In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

##Iniciando o processo dados
navegador.get('https://statusinvest.com.br/acoes/busca-avancada/')
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[3]/div/div/div/button[2]').click()
time.sleep(2)
navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[4]/div/div[1]/div[2]/a/span').click()
time.sleep(2)

#Fecha o navegador
navegador.quit()

## Renomeia o arquivo e move para pasta desejada
source = '/Users/scheiner/Downloads/statusinvest-busca-avancada.csv'
dest = '/Users/scheiner/Documents/GitHub/StatusInvest/stocks.csv'
os.rename(source, dest) 

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [3]:
### Tratamento dos dados importados:

df = pd.read_csv("stocks.csv", sep=';', decimal=',', thousands='.', encoding='ISO-8859-1' )
    
df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
df['UP/DOWNSIDE'] = (df[' VPA'] / df['PRECO'])-1
    
valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]

valuation.fillna(value=0, inplace=True)
valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
valuation = valuation.loc[valuation['DY'] >= 6].loc[
    valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
    valuation['ROE'] >= 6].loc[
    valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
    valuation['LIQ.CORR'] >= 0.5]


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
    

        #Criando dict
        dicionario = {  "stocks": stocks, 
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [47]:
### Preparando os dados e Dataframe

list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')
list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')
list_stocks.to_csv('list_stocks.csv')

list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)

list_stocks = list_stocks.replace('AcessÃ³rios', 'Acessorios')
list_stocks = list_stocks.replace('Armas e MuniÃ§Ãµes', 'Armas e Municoes')
list_stocks = list_stocks.replace('Artefatos de Ferro e AÃ§o', 'Artefatos de Ferro e Aco')
list_stocks = list_stocks.replace('AutomÃ³veis e Motocicletas', 'Automoveis e Motocicletas')
list_stocks = list_stocks.replace('AÃ§ucar e Alcool', 'Acucar e Alcool')
list_stocks = list_stocks.replace('CalÃ§ados', 'Calcados')
list_stocks = list_stocks.replace('ConstruÃ§Ã£o Pesada', 'Construcao Pesada')
list_stocks = list_stocks.replace('EletrodomÃ©sticos', 'Eletrodomesticos')
list_stocks = list_stocks.replace('Energia ElÃ©trica', 'Energia Eletrica')
list_stocks = list_stocks.replace('Equipamentos e ServiÃ§os', 'Equipamentos e Servicos')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de ImÃ³veis', 'Exploracao de Imoveis')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de Rodovias', 'Exploracao de Rodovias')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o. Refino e DistribuiÃ§Ã£o', 'Exploracao Refino e Distribuicao')
list_stocks = list_stocks.replace('GestÃ£o de Recursos e Investimentos', 'Gestao Recursos e Investimentos')
list_stocks = list_stocks.replace('GÃ¡s', 'Gas')
list_stocks = list_stocks.replace('IncorporaÃ§Ãµes', 'Incorporacoes')
list_stocks = list_stocks.replace('IntermediaÃ§Ã£o ImobiliÃ¡ria', 'Intermediacao Imobiliaria')
list_stocks = list_stocks.replace('LogÃ­stica', 'Logistica')
list_stocks = list_stocks.replace('Material AeronÃ¡utico e de Defesa', 'Material Aeronautico e Defesa')
list_stocks = list_stocks.replace('Material RodoviÃ¡rio', 'Material Rodoviario')
list_stocks = list_stocks.replace('Minerais MetÃ¡licos', 'Minerais Metalicos')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. ConstruÃ§Ã£o e AgrÃ­colas', 'Maq Equip Construcao e Agricolas')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. Industriais', 'Maq Equip Industriais')
list_stocks = list_stocks.replace('MÃ³veis', 'Moveis')
list_stocks = list_stocks.replace('PetroquÃ­micos', 'Petroquimicos')
list_stocks = list_stocks.replace('Produtos para ConstruÃ§Ã£o', 'Produtos para Construcao')
list_stocks = list_stocks.replace('ProduÃ§Ã£o de Eventos e Shows', 'Producao de Eventos e Shows')
list_stocks = list_stocks.replace('ProduÃ§Ã£o e DifusÃ£o de Filmes e Programas', 'Producao Difusao de Filmes e Programas')
list_stocks = list_stocks.replace('Programas de FidelizaÃ§Ã£o', 'Programas de Fidelizacao')
list_stocks = list_stocks.replace('Programas e ServiÃ§os', 'Programas e Servicos')
list_stocks = list_stocks.replace('QuÃ­micos Diversos', 'Quimicos Diversos')
list_stocks = list_stocks.replace('Serv.MÃ©d.Hospit..AnÃ¡lises e DiagnÃ³sticos', 'Serv Mao Hosp Analises e Diagnosticos')
list_stocks = list_stocks.replace('ServiÃ§os Diversos', 'Servicos Diversos')
list_stocks = list_stocks.replace('ServiÃ§os Educacionais', 'Servicos Educacionais')
list_stocks = list_stocks.replace('ServiÃ§os Financeiros Diversos', 'Servicos Financeiros Diversos')
list_stocks = list_stocks.replace('Soc. CrÃ©dito e Financiamento', 'Soc Credito e Financiamento')
list_stocks = list_stocks.replace('Tecidos. VestuÃ¡rio e CalÃ§ados', 'Tecidos Vestuario e Calcados')
list_stocks = list_stocks.replace('TelecomunicaÃ§Ãµes', 'Telecomunicacoes')
list_stocks = list_stocks.replace('Transporte AÃ©reo', 'Transporte Aereo')
list_stocks = list_stocks.replace('Transporte FerroviÃ¡rio', 'Transporte Ferroviario')
list_stocks = list_stocks.replace('Transporte HidroviÃ¡rio', 'Transporte Hidroviario')
list_stocks = list_stocks.replace('UtensÃ­lios DomÃ©sticos', 'Utensilios Domesticos')
list_stocks = list_stocks.replace('VestuÃ¡rio', 'Vestuario')
list_stocks = list_stocks.replace('Ãgua e Saneamento', 'Agua e Saneamento')

valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)

list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)

investing = pd.merge(valuation, list_stocks, on='TICKERS')

value_invest = investing[['TICKERS', 'PRECO', 'MIN_MES', 'MAX_MES', 'VAL_MES', 'MIN_52S', 'MAX_52S',  'VAL_12M', 
                       ' VPA', 'DY', ' LPA', 'P/L' ,'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 'P/VP', 'EV/EBIT', 
                       'LIQ.CORR', 'MARG.LIQ', 'DIV.LIQ/PL', 'SEGMENTO']]

### PEGANDO O INDICE IBOVESPA NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

t = get_tickers('IBOV')
ibov = pd.DataFrame(t)
ibov['IBOVESPA'] = 'IBOV'
ibov.rename(columns={0: "TICKERS" }, inplace=True)
ibov = ibov


### PEGANDO O INDICE SMALL CAPS NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

u = get_tickers('SMLL')

smll = pd.DataFrame(u)
smll['SMALLCAP'] = 'SMLL'
smll.rename(columns={0: "TICKERS" }, inplace=True)
smll = smll

### PEGANDO O INDICE SMALL IDIV NA B3
def get_tickers(indice):
  url = 'http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice={}&idioma=pt-br'.format(indice.upper())
  return (pd.read_html(url, decimal=',', thousands='.', index_col='Código')[0][:-1].index).to_list()

v = get_tickers('IDIV')

idiv = pd.DataFrame(v)
idiv['DIVIDENDOS'] = 'IDIV'
idiv.rename(columns={0: "TICKERS" }, inplace=True)
idiv = idiv

value_invest = pd.merge(value_invest, ibov, on='TICKERS', how = 'left').fillna(0)

value_invest = pd.merge(value_invest, smll, on='TICKERS', how = 'left').fillna(0)

value_invest = pd.merge(value_invest, idiv, on='TICKERS', how = 'left').fillna(0)

value_invest['IBOVESPA'] = (value_invest['IBOVESPA']).str.strip('0')
value_invest['SMALLCAP'] = (value_invest['SMALLCAP']).str.strip('0')
value_invest['DIVIDENDOS'] = (value_invest['DIVIDENDOS']).str.strip('0')

value_invest.fillna('', inplace=True)

value_invest = value_invest.loc[value_invest['DY'] >= 6].loc[
    value_invest['P/L'] >= 0.01].loc[value_invest['P/L'] < 15].loc[
    value_invest['ROE'] >= 6].loc[
    value_invest['EV/EBIT'] >= 0.01].loc[value_invest['EV/EBIT'] <= 15].loc[
    value_invest['LIQ.CORR'] >= 0.5].loc[
    value_invest['UP/DOWNSIDE'] >= 0.1]#.loc[
    # value_invest['IBOVESPA'] == 'IBOV'].loc[
    # value_invest['SMALLCAP'] == 'SMLL'].loc[
    # value_invest['DIVIDENDOS'] == 'IDIV']


In [48]:
value_invest

,TICKERS,PRECO,MIN_MES,MAX_MES,VAL_MES,MIN_52S,MAX_52S,VAL_12M,VPA,DY,...,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL,SEGMENTO,IBOVESPA,SMALLCAP,DIVIDENDOS
0,ABCB4,20.13,18.35,20.13,7.82,15.27,21.24,7.47,23.98,7.87,...,14.50,0.83,4.36,1.50,19.35,0.00,Bancos,,SMLL,IDIV
3,BBAS3,47.36,46.35,47.36,0.49,29.19,50.89,30.22,56.75,9.71,...,20.01,0.83,3.18,12.76,12.35,0.00,Bancos,IBOV,,IDIV
4,BBDC3,13.20,12.80,13.20,0.08,11.03,16.50,-11.35,15.64,6.91,...,9.78,0.85,10.97,7.16,7.37,0.00,Bancos,IBOV,,
7,BGIP3,26.00,24.90,28.99,4.42,21.06,33.22,1.05,35.35,8.93,...,10.32,0.71,6.38,0.76,5.27,0.00,Bancos,,,
8,BGIP4,24.00,22.12,24.10,8.50,15.08,27.39,42.01,35.35,10.24,...,10.32,0.68,6.38,0.76,5.27,0.00,Bancos,,,
11,BRSR3,12.90,12.65,13.50,-0.39,9.69,14.88,7.41,23.36,7.31,...,8.61,0.55,5.81,2.81,5.73,0.00,Bancos,,,
12,BRSR6,12.48,12.25,12.91,-0.64,8.66,15.18,14.81,23.36,7.57,...,8.61,0.53,5.81,2.81,5.73,0.00,Bancos,,SMLL,IDIV
13,CATA3,103.61,103.61,113.46,0.00,0.00,0.00,0.00,1214.17,38.09,...,17.46,0.09,1.91,1.85,9.25,0.27,Fios e Tecidos,,,
17,CIEL3,3.63,3.63,3.86,-5.96,3.63,5.69,-28.12,4.31,8.01,...,16.24,0.84,4.77,1.19,17.91,0.55,Servicos Financeiros Diversos,IBOV,SMLL,IDIV
18,CLSC3,61.99,60.41,63.00,1.46,42.93,64.49,35.23,82.99,6.43,...,19.14,0.75,4.20,0.95,5.98,0.45,Energia Eletrica,,,
